In [ ]:
!pip install mediapipe opencv-python-headless numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found exi

In [ ]:
!pip install -U google-genai  # Python SDK for Gemini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.29.0
    Uninstalling google-genai-1.29.0:
      Successfully uninstalled google-genai-1.29.0


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import json
import os
import math
import matplotlib.pyplot as plt

from google.colab import files
from google.genai import Client
from google.genai import types

In [ ]:
import os

os.environ["GEMINI_API_KEY"] = "your-gemini-api-key"

In [ ]:
client = Client(api_key=os.environ["GEMINI_API_KEY"])

In [ ]:
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

VIDEO_PATH = "input_video.mp4"
OUTPUT_VIDEO = os.path.join(OUTPUT_DIR, "annotated_video.mp4")
METRICS_FILE = os.path.join(OUTPUT_DIR, "metrics_log.json")
EVAL_FILE = os.path.join(OUTPUT_DIR, "evaluation.json")
OUTPUT_PS = os.path.join(OUTPUT_DIR, 'output_phase_seg.mp4')
SWING_VIDEO_FILE = os.path.join(OUTPUT_DIR, "bat_swing_analysis.mp4")
REPORT_FILE = os.path.join(OUTPUT_DIR, "biomechanics_report.png")

In [ ]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

## **Utility Functions**

In [ ]:
def calculate_angle(a, b, c):
    """
    Calculates the angle between three points (a, b, c) in degrees.
    Point b is the vertex of the angle.
    """
    a, b, c = np.array(a), np.array(b), np.array(c)
    dot_product = np.dot(a - b, c - b)
    norm_product = np.linalg.norm(a - b) * np.linalg.norm(c - b)
    if norm_product == 0:
        return 0.0
    cosine = dot_product / (norm_product + 1e-6)
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

In [ ]:
def generate_feedback_with_gemini(scores):
    """
    Generates AI-powered coaching feedback using the Gemini API.
    This function will be much more effective with the improved metrics.
    """
    # Assuming the API client is already initialized
    client = Client(api_key=os.environ["GEMINI_API_KEY"])

    prompt = (
        "You’re an expert cricket batting coach. "
        "Here are the shot biomechanics scores and data:\\n"
        f"Average Spine Lean (relative to vertical): {scores.get('Balance', 'N/A')} degrees\\n"
        f"Average Head Horizontal Movement: {scores.get('Head Position', 'N/A')} pixels\\n"
        f"Average Elbow Angle: {scores.get('Swing Control', 'N/A')} degrees\\n"
        f"Average Foot Position (relative to start): {scores.get('Footwork', 'N/A')}\\n\\n"
        "Please provide 1-2 clear, specific lines of constructive feedback on each category."
        "Give actionable advice based on these metrics."
    )

    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents=prompt
    )
    return response.text

## **Analyze video**

In [ ]:
def analyze_video(video_path):
    """
    Analyzes a video of a cricket batter and calculates biomechanical metrics.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(
        OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height)
    )

    metrics_log = {
        "elbow_angles": [],
        "spine_leans": [],
        "head_horizontal_movement": [],
        "foot_positions_x": [],
    }

    prev_landmarks = None

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark

                def get_point(idx):
                    return [lm[idx].x * width, lm[idx].y * height]

                left_shoulder = get_point(mp_pose.PoseLandmark.LEFT_SHOULDER.value)
                left_elbow = get_point(mp_pose.PoseLandmark.LEFT_ELBOW.value)
                left_wrist = get_point(mp_pose.PoseLandmark.LEFT_WRIST.value)
                left_hip = get_point(mp_pose.PoseLandmark.LEFT_HIP.value)
                left_ankle = get_point(mp_pose.PoseLandmark.LEFT_ANKLE.value)
                head = get_point(mp_pose.PoseLandmark.NOSE.value)

                elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                metrics_log["elbow_angles"].append(elbow_angle)

                vertical_point = [left_shoulder[0], left_shoulder[1] - 100]
                spine_lean = calculate_angle(left_hip, left_shoulder, vertical_point)
                metrics_log["spine_leans"].append(spine_lean)

                if prev_landmarks:
                    prev_head_x = prev_landmarks.landmark[mp_pose.PoseLandmark.NOSE.value].x * width
                    head_movement = abs(head[0] - prev_head_x)
                    metrics_log["head_horizontal_movement"].append(head_movement)

                metrics_log["foot_positions_x"].append(left_ankle[0])
                prev_landmarks = results.pose_landmarks

                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                )

                cv2.putText(image, f"Elbow: {int(elbow_angle)} deg", (20, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                cv2.putText(image, f"Spine Lean: {int(spine_lean)} deg", (20, 70),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 200, 0), 2)
                if metrics_log["head_horizontal_movement"]:
                    current_head_move = metrics_log["head_horizontal_movement"][-1]
                    cv2.putText(image, f"Head Move: {current_head_move:.2f} px", (20, 100),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 200, 255), 2)

            out.write(image)

    cap.release()
    out.release()

    with open(METRICS_FILE, "w") as f:
        json.dump(metrics_log, f, indent=2)

    eval_data = {
        "Footwork": round(np.mean(metrics_log["foot_positions_x"]), 2) if metrics_log["foot_positions_x"] else 0,
        "Head Position": round(np.mean(metrics_log["head_horizontal_movement"]), 2) if metrics_log["head_horizontal_movement"] else 0,
        "Swing Control": round(np.mean(metrics_log["elbow_angles"]), 2) if metrics_log["elbow_angles"] else 0,
        "Balance": round(np.mean(metrics_log["spine_leans"]), 2) if metrics_log["spine_leans"] else 0,
        "Follow-through": "Needs custom logic"
    }

    feedback_text = generate_feedback_with_gemini(eval_data)
    eval_data["AI_Feedback"] = feedback_text

    with open(EVAL_FILE, "w") as f:
        json.dump(eval_data, f, indent=2)

    print(f"Saved: {OUTPUT_VIDEO}, {METRICS_FILE}, {EVAL_FILE}")

##  **Automatic Phase Segmentation**

In [ ]:
def phase_segmentation(video_path, output_video_path):
    # Heuristics and state for phase detection
    BOWLER_DELIVERY_THRESHOLD = 5.0
    BATSMAN_DOWNSWING_THRESHOLD = 5.0
    IMPACT_ELBOW_THRESHOLD = 110.0
    FOLLOW_THROUGH_THRESHOLD = 10.0
    RECOVERY_THRESHOLD = 5.0
    SLOW_DOWN_FACTOR = 3

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(
        output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height)
    )

    phase = "Stance"
    prev_batsman_wrist_y = None
    prev_bowler_wrist_y = None

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark

                def get_point(idx):
                    return [lm[idx].x * width, lm[idx].y * height]

                batsman_wrist = get_point(mp_pose.PoseLandmark.LEFT_WRIST.value)
                batsman_elbow = get_point(mp_pose.PoseLandmark.LEFT_ELBOW.value)
                batsman_shoulder = get_point(mp_pose.PoseLandmark.LEFT_SHOULDER.value)

                bowler_wrist = get_point(mp_pose.PoseLandmark.RIGHT_WRIST.value)

                if prev_batsman_wrist_y is not None and prev_bowler_wrist_y is not None:
                    batsman_wrist_movement_y = batsman_wrist[1] - prev_batsman_wrist_y
                    bowler_wrist_movement_y = bowler_wrist[1] - prev_bowler_wrist_y

                    if phase == "Stance":
                        if bowler_wrist_movement_y > BOWLER_DELIVERY_THRESHOLD:
                            phase = "Pre-Swing"
                    elif phase == "Pre-Swing":
                        if batsman_wrist_movement_y > BATSMAN_DOWNSWING_THRESHOLD:
                            phase = "Downswing"
                    elif phase == "Downswing":
                        elbow_angle = calculate_angle(batsman_shoulder, batsman_elbow, batsman_wrist)
                        if elbow_angle < IMPACT_ELBOW_THRESHOLD and batsman_wrist_movement_y > FOLLOW_THROUGH_THRESHOLD:
                            phase = "Impact/Follow-through"
                    elif phase == "Impact/Follow-through":
                        if batsman_wrist_movement_y > FOLLOW_THROUGH_THRESHOLD:
                            phase = "Recovery"
                    elif phase == "Recovery":
                        if abs(batsman_wrist_movement_y) < RECOVERY_THRESHOLD:
                            phase = "Stance"

                prev_batsman_wrist_y = batsman_wrist[1]
                prev_bowler_wrist_y = bowler_wrist[1]

                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                )

                cv2.putText(image, f"Phase: {phase}", (20, 130),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

            if phase in ["Downswing", "Impact/Follow-through"]:
                for _ in range(SLOW_DOWN_FACTOR):
                    out.write(image)
            else:
                out.write(image)

    cap.release()
    out.release()
    print(f"Saved segmented video to: {output_video_path}")

##  **Basic Bat Detection/Tracking**

In [ ]:
def analyze_bat_path(video_path, output_path):
    """
    Analyzes the video to approximate bat movement and swing path.
    It uses the player's pose landmarks to create a bat line and visualizes its angle.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(
        output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height)
    )

    swing_angles = []

    # Heuristics and state for phase detection
    BOWLER_DELIVERY_THRESHOLD = 5.0
    BATSMAN_DOWNSWING_THRESHOLD = 5.0
    SLOW_DOWN_FACTOR = 3

    phase = "Stance"
    prev_batsman_wrist_y = None
    prev_bowler_wrist_y = None

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark

                def get_point(idx):
                    return [lm[idx].x * width, lm[idx].y * height]

                batsman_wrist = get_point(mp_pose.PoseLandmark.LEFT_WRIST.value)
                batsman_elbow = get_point(mp_pose.PoseLandmark.LEFT_ELBOW.value)
                batsman_shoulder = get_point(mp_pose.PoseLandmark.LEFT_SHOULDER.value)

                bowler_wrist = get_point(mp_pose.PoseLandmark.RIGHT_WRIST.value)

                bat_length_factor = 2.0
                bat_tip_x = batsman_wrist[0] + (batsman_wrist[0] - batsman_elbow[0]) * bat_length_factor
                bat_tip_y = batsman_wrist[1] + (batsman_wrist[1] - batsman_elbow[1]) * bat_length_factor
                bat_tip = [bat_tip_x, bat_tip_y]

                vertical_point = [batsman_wrist[0], batsman_wrist[1] - 100]
                swing_angle = calculate_angle(bat_tip, batsman_wrist, vertical_point)
                swing_angles.append(swing_angle)

                if prev_batsman_wrist_y is not None and prev_bowler_wrist_y is not None:
                    batsman_wrist_movement_y = batsman_wrist[1] - prev_batsman_wrist_y
                    bowler_wrist_movement_y = bowler_wrist[1] - prev_bowler_wrist_y

                    if phase == "Stance":
                        if bowler_wrist_movement_y > BOWLER_DELIVERY_THRESHOLD:
                            phase = "Pre-Swing"
                    elif phase == "Pre-Swing":
                        if batsman_wrist_movement_y > BATSMAN_DOWNSWING_THRESHOLD:
                            phase = "Downswing"
                    elif phase == "Downswing":
                        if batsman_wrist_movement_y < 1.0:
                            phase = "Recovery"
                    elif phase == "Recovery":
                        if abs(batsman_wrist_movement_y) < 1.0:
                            phase = "Stance"

                prev_batsman_wrist_y = batsman_wrist[1]
                prev_bowler_wrist_y = bowler_wrist[1]

                cv2.line(image, (int(batsman_wrist[0]), int(batsman_wrist[1])), (int(bat_tip[0]), int(bat_tip[1])), (0, 255, 255), 2)
                cv2.circle(image, (int(bat_tip[0]), int(bat_tip[1])), 10, (0, 255, 255), -1)

                text = f"Swing Angle: {int(swing_angle)} deg"
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.8
                font_thickness = 2
                text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
                text_w, text_h = text_size
                cv2.rectangle(image, (20, 80), (20 + text_w, 80 + text_h + 5), (0, 0, 0), -1)
                cv2.putText(image, text, (20, 80 + text_h), font, font_scale, (255, 255, 255), font_thickness)

                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                )

            if phase in ["Pre-Swing", "Downswing"]:
                for _ in range(SLOW_DOWN_FACTOR):
                    out.write(image)
            else:
                out.write(image)

    cap.release()
    out.release()

    print(f"Saved bat swing analysis video to: {output_path}")

    if swing_angles:
        avg_swing_angle = np.mean(swing_angles)
        print(f"Average swing angle during the video: {avg_swing_angle:.2f} degrees")
    else:
        print("Could not calculate swing angles.")


## **Complete Report**

In [ ]:
def create_analysis_report(metrics_file, report_file):
    """
    Creates a visual report of the player's biomechanics over time and saves it as a PNG.
    """
    with open(metrics_file, 'r') as f:
        metrics_log = json.load(f)

    elbow_angles = metrics_log.get('elbow_angles', [])
    spine_leans = metrics_log.get('spine_leans', [])

    if not elbow_angles or not spine_leans:
        print("Error: Metrics data is empty. Cannot create report.")
        return

    frames = np.arange(len(elbow_angles))

    # Calculate smoothness metrics (frame-to-frame deltas)
    elbow_deltas = np.diff(elbow_angles)
    spine_deltas = np.diff(spine_leans)

    # Create the plot
    plt.style.use('dark_background')
    fig, ax1 = plt.subplots(figsize=(12, 6))

    color_elbow = 'tab:blue'
    ax1.set_xlabel('Frame Number')
    ax1.set_ylabel('Elbow Angle (deg)', color=color_elbow)
    ax1.plot(frames, elbow_angles, color=color_elbow, label='Elbow Angle')
    ax1.tick_params(axis='y', labelcolor=color_elbow)
    ax1.grid(True, linestyle='--', alpha=0.6)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    color_spine = 'tab:orange'
    ax2.set_ylabel('Spine Lean (deg)', color=color_spine)
    ax2.plot(frames, spine_leans, color=color_spine, label='Spine Lean')
    ax2.tick_params(axis='y', labelcolor=color_spine)

    plt.title('Player Biomechanics Over Time', color='white')
    fig.tight_layout()
    fig.legend(loc='upper right', bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)

    plt.savefig(report_file, dpi=100)
    print(f"Saved analysis report to: {report_file}")
    plt.close(fig)


## **Main Function**

In [ ]:
def main():
    """
    Main function to orchestrate all video analysis tasks.
    """
    print("Starting video analysis...")

    # 1. Biomechanical Analysis
    print("Running biomechanical analysis...")
    analyze_video(VIDEO_PATH)

    print("\n---")

    # 2. Phase Segmentation
    print("Running phase segmentation...")
    phase_segmentation(VIDEO_PATH, OUTPUT_PS)

    print("\n---")

    # 3. Bat Path Analysis
    print("Running bat path analysis...")
    analyze_bat_path(VIDEO_PATH, SWING_VIDEO_FILE)

    print("\n---")

    # 4. Analysis Report
    print("Generating biomechanics report...")
    create_analysis_report(METRICS_FILE, REPORT_FILE)

    print("\nVideo analysis complete. Check the 'output' folder for results.")

# --- Call the main function ---
if __name__ == '__main__':
    main()


Starting video analysis...
Running biomechanical analysis...
Saved: output/annotated_video.mp4, output/metrics_log.json, output/evaluation.json

---
Running phase segmentation...
Saved segmented video to: output/output_phase_seg.mp4

---
Running bat path analysis...
Saved bat swing analysis video to: output/bat_swing_analysis.mp4
Average swing angle during the video: 120.41 degrees

---
Generating biomechanics report...
Saved analysis report to: output/biomechanics_report.png

Video analysis complete. Check the 'output' folder for results.
